In [ ]:
# INSTALL MODULES
%pip install pickle5

# For ray.Tune
%pip install ray torch torchvision      # Preferred install command when running PyTorch
#%pip install -U ray                    # Alternate installation for Tune

# For Tensorboard
%pip install tensorboardX

# IMPORTS
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
import os
import pickle5 as pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
from ray import tune
import tensorboardX

# Load the TensorBoard notebook extension
%load_ext tensorboard

# Mount Google Drive in Colab
#from google.colab import files
from google.colab import drive
drive.mount("/content/gdrive")

# Import modules in Colab from another notebooks
%run '/content/gdrive/MyDrive/Colab Notebooks/dataset.ipynb'
%run '/content/gdrive/MyDrive/Colab Notebooks/models.ipynb'

if not torch.cuda.is_available():
    raise RuntimeError("You should enable GPU runtime.")



In [ ]:
# To speed up training, it's better to copy dataset from Drive to a Colab folder

# choose a local (colab) directory to store the data.
local_dataset_path = os.path.expanduser('/content/data')
try:
  os.makedirs(local_dataset_path)
except: pass

dataset_path = '/content/gdrive/MyDrive/jester_dataset/'

!cp -avr "{dataset_path}" "{local_dataset_path}"

# Make sure it's there
!ls -lha /content/gdrive/MyDrive/jester_dataset/features

'/content/gdrive/MyDrive/jester_dataset/' -> '/content/data/jester_dataset'
'/content/gdrive/MyDrive/jester_dataset/features' -> '/content/data/jester_dataset/features'
'/content/gdrive/MyDrive/jester_dataset/features/features_flow.pickle' -> '/content/data/jester_dataset/features/features_flow.pickle'
'/content/gdrive/MyDrive/jester_dataset/features/features_bad_videos.pickle' -> '/content/data/jester_dataset/features/features_bad_videos.pickle'
'/content/gdrive/MyDrive/jester_dataset/features/features_RGB.pickle' -> '/content/data/jester_dataset/features/features_RGB.pickle'
'/content/gdrive/MyDrive/jester_dataset/csv' -> '/content/data/jester_dataset/csv'
'/content/gdrive/MyDrive/jester_dataset/csv/validation.csv' -> '/content/data/jester_dataset/csv/validation.csv'
'/content/gdrive/MyDrive/jester_dataset/csv/train.csv' -> '/content/data/jester_dataset/csv/train.csv'
'/content/gdrive/MyDrive/jester_dataset/csv/labels.csv' -> '/content/data/jester_dataset/csv/labels.csv'
'/content/gd

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Retrieves features from i3d_resnet50_v1_kinetics400 Gluon pre-trained model extracted from our 9 classes' Jester dataset videos
with open('/content/data/jester_dataset/features/features_RGB.pickle', 'rb') as handle:
    features_dict = pickle.load(handle)        # dict as {n_video:features, ...}  features: (1,2048)
    
csv_dir = '/content/data/jester_dataset/csv/'  # if in Google Cloud --> csv_dir = '/mnt/disks/disk-1/jester_dataset/dataset/csvs/' 
train_csv = csv_dir + 'train.csv' 
val_csv = csv_dir + 'validation.csv'
labels = csv_dir + 'labels.csv'

In [ ]:
def train_epoch(model, train_loader, optimizer, criterion, epoch):  
  model.train()
  accs, losses = [], []

  for features, labels in train_loader:
    optimizer.zero_grad()
    features, labels = features.to(device), labels.to(device)
    output = model(features)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    accs.append(accuracy(labels, output))
    losses.append(loss.item())

  return np.mean(losses), np.mean(accs)

In [ ]:
def eval_epoch(model, val_loader, criterion, epoch): 
  with torch.no_grad():
    model.eval()
    accs, losses = [], []

    for features, labels in val_loader:
      features, labels = features.to(device), labels.to(device)
      output = model(features)
      loss = criterion(output, labels)
      accs.append(accuracy(labels, output))
      losses.append(loss.item())

    return np.mean(losses), np.mean(accs)

In [ ]:
def accuracy(labels, outputs):
    preds = outputs.argmax(-1)
    acc = (preds == labels.view_as(preds)).cpu().float().detach().numpy().mean()
    return acc

In [ ]:
def train_model(config):
  # DATASETS
  train_dataset = JesterDatasetOneStream(features_dict, train_csv, labels)
  validation_dataset = JesterDatasetOneStream(features_dict, val_csv, labels)

  train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
  validation_loader = DataLoader(validation_dataset, batch_size=config['batch_size'], shuffle=False)

  # MODEL
  model = ClassifierOneStream().to(device)
  optimizer = optim.Adam(model.parameters(), lr = config['lr'])

  counts = pd.read_csv(train_csv).Action.value_counts()       # Counts = list of video counts for each class
  weights = torch.tensor([max(counts)/x for x in counts])     # calculates weights for all classes in training dataset (weight regularization)
  criterion = nn.CrossEntropyLoss(weight=weights).to(device)  # assigns weight to each of the classes. This is particularly useful when you have an unbalanced training set
  
  train_losses = []
  val_losses = []
  train_accs = []
  val_accs = []
  
  for epoch in range(config['epochs']):
    
    loss, acc = train_epoch(model, train_loader, optimizer, criterion, epoch)
    train_losses.append(loss)
    train_accs.append(acc)
    print(f"Train Epoch {epoch} loss={loss:.2f} acc={acc:.2f}")
        
    loss, acc = eval_epoch(model, validation_loader, criterion, epoch)
    val_losses.append(loss)
    val_accs.append(acc)
    print(f"Eval Epoch {epoch} loss={loss:.2f} acc={acc:.2f}")
    tune.report(tune_loss=loss, accuracy=acc)       # send report to Tune

  # Saves the model_state_dict to load it later in the final system's model to do inference 
  savedir = '/content/gdrive/MyDrive/model/state_dict.pt'
  print(f"Saving checkpoint to {savedir}...")
  checkpoint = {
      "model_state_dict": model.cpu().state_dict(),
  }
  torch.save(checkpoint, savedir)
  
  #return model, [train_losses, train_accs, val_losses, val_accs] # Model can't be returned by trainable function when using Tune
  #return [train_losses, train_accs, val_losses, val_accs]         

In [ ]:
if __name__ == "__main__":

    config = {
            "lr": tune.loguniform(1e-4, 1e-2),
            "batch_size": tune.choice([16, 32, 64, 128]),
            "epochs": 20,
            }
    
    analysis = tune.run(                        # run hyperparameter tuning trials with ray.Tune
        train_model,
        metric="tune_loss",
        mode="min",
        num_samples=5,
        resources_per_trial= {
            #"cpu": 1,
            "gpu": 1  # set this for GPUs
        },
        config = config
        )

    print("Best hyperparameters found were: ", analysis.best_config)

    print("Hyperparameter tuning finished")

    

2021-03-11 19:55:36,402	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
2021-03-11 19:55:39,226	WARNING function_runner.py:541 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-03-11 19:55:53,491	WARNING worker.py:1107 -- Warning: The actor ImplicitFunc has size 396297732 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-03-11 19:55:53,950	WARNING util.py:152 -- The `start_trial` operation took 6.870 s, which may be a performance bottleneck.


Trial name,status,loc,batch_size,lr
train_model_c3a89_00000,RUNNING,,128,0.000988528


(pid=459) Train Epoch 0 loss=1.30 acc=0.52
Result for train_model_c3a89_00000:
  accuracy: 0.6309525370597839
  date: 2021-03-11_19-56-33
  done: false
  experiment_id: 4d2f3f3a3ffe483fba9102d755ceee09
  hostname: 49f8f80687ca
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 459
  time_since_restore: 34.79302978515625
  time_this_iter_s: 34.79302978515625
  time_total_s: 34.79302978515625
  timestamp: 1615492593
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: c3a89_00000
  tune_loss: 1.0045657144036404
  


Trial name,status,loc,batch_size,lr,iter,total time (s),tune_loss,accuracy
train_model_c3a89_00000,RUNNING,172.28.0.2:459,128,0.000988528,1,34.793,1.00457,0.630953
train_model_c3a89_00001,PENDING,,128,0.00840321,,,,


(pid=459) Eval Epoch 0 loss=1.00 acc=0.63
(pid=459) Train Epoch 1 loss=1.05 acc=0.61
Result for train_model_c3a89_00000:
  accuracy: 0.6637483835220337
  date: 2021-03-11_19-56-56
  done: false
  experiment_id: 4d2f3f3a3ffe483fba9102d755ceee09
  hostname: 49f8f80687ca
  iterations_since_restore: 2
  node_ip: 172.28.0.2
  pid: 459
  time_since_restore: 58.585206031799316
  time_this_iter_s: 23.792176246643066
  time_total_s: 58.585206031799316
  timestamp: 1615492616
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: c3a89_00000
  tune_loss: 0.9135019682174506
  


...

Trial name,status,loc,batch_size,lr,iter,total time (s),tune_loss,accuracy
train_model_c3a89_00004,RUNNING,172.28.0.2:1021,128,0.000496012,20,477.417,0.697419,0.731973
train_model_c3a89_00000,TERMINATED,,128,0.000988528,20,484.647,0.712474,0.724416
train_model_c3a89_00001,TERMINATED,,128,0.00840321,20,472.643,0.831419,0.687414
train_model_c3a89_00002,TERMINATED,,64,0.000365134,20,481.78,0.684731,0.741481
train_model_c3a89_00003,TERMINATED,,128,0.00286304,20,482.919,0.761491,0.707082


(pid=1021) Eval Epoch 19 loss=0.70 acc=0.73
(pid=1021) Saving checkpoint to /content/gdrive/MyDrive/model/state_dict.pt...
Result for train_model_c3a89_00004:
  accuracy: 0.7319730520248413
  date: 2021-03-11_20-36-33
  done: true
  experiment_id: d0c5dbc8cf114f87aa9151746d1fb2fe
  experiment_tag: 4_batch_size=128,lr=0.00049601
  hostname: 49f8f80687ca
  iterations_since_restore: 20
  node_ip: 172.28.0.2
  pid: 1021
  time_since_restore: 477.41713976860046
  time_this_iter_s: 23.83824849128723
  time_total_s: 477.41713976860046
  timestamp: 1615494993
  timesteps_since_restore: 0
  training_iteration: 20
  trial_id: c3a89_00004
  tune_loss: 0.6974188470563223
  


Trial name,status,loc,batch_size,lr,iter,total time (s),tune_loss,accuracy
train_model_c3a89_00000,TERMINATED,,128,0.000988528,20,484.647,0.712474,0.724416
train_model_c3a89_00001,TERMINATED,,128,0.00840321,20,472.643,0.831419,0.687414
train_model_c3a89_00002,TERMINATED,,64,0.000365134,20,481.78,0.684731,0.741481
train_model_c3a89_00003,TERMINATED,,128,0.00286304,20,482.919,0.761491,0.707082
train_model_c3a89_00004,TERMINATED,,128,0.000496012,20,477.417,0.697419,0.731973


2021-03-11 20:36:33,897	INFO tune.py:450 -- Total run time: 2458.03 seconds (2449.98 seconds for the tuning loop).


Best hyperparameters found were:  {'lr': 0.00036513370013530944, 'batch_size': 64, 'epochs': 20}
Hyperparameter tuning finished


In [ ]:
%tensorboard --logdir ~/ray_results